# [Introduction to Relational Databases in SQL](https://www.datacamp.com/courses/introduction-to-relational-databases-in-sql)

## First

We look at the first couple chapters with the Python sqlite3 package.

## Second

We look at the entire course using SQLalchemy and postgres SQL.

In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('sqlite/test.db')

In [3]:
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE professors (
firstname text,
lastname text
);''')

# Add a column to the professors table
c.execute('''ALTER TABLE professors
ADD COLUMN university_shortname text;''')

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [4]:
conn = sqlite3.connect('sqlite/test.db')

In [5]:
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE affiliations (
organisation text);
''')

# Rename a column
c.execute('''ALTER TABLE affiliations
RENAME COLUMN organisation TO organization;''')

# Save (commit) the changes
conn.commit()

In [6]:
columns = ['firstname', 
           'lastname', 
           'university', 
           'university_shortname', 
           'university_city', 
           'function', 
           'organization', 
           'organization_sector']

uni_prof_df = pd.read_csv('uni_prof.csv', names=columns)

uni_prof_df.head()

,firstname,lastname,university,university_shortname,university_city,function,organization,organization_sector
0,Karl,Aberer,ETHLausanne,EPF,Lausanne,ChairmanofL3SAdvisoryBoard,L3SAdvisoryBoard,Education&research
1,Karl,Aberer,ETHLausanne,EPF,Lausanne,MemberConseilofZeno-KarlSchindlerFoundation,Zeno-KarlSchindlerFoundation,Education&research
2,Karl,Aberer,ETHLausanne,EPF,Lausanne,MemberofConseilFondationIDIAP,FondationIDIAP,Education&research
3,Karl,Aberer,ETHLausanne,EPF,Lausanne,PanelMember,SNFAmbizioneProgram,Education&research
4,RezaShokrollah,Abhari,ETHZürich,ETH,Zurich,Aufsichtsratsmandat,PNEWindAG,Energy


In [7]:
uni_prof_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 8 columns):
firstname               99 non-null object
lastname                99 non-null object
university              99 non-null object
university_shortname    99 non-null object
university_city         99 non-null object
function                75 non-null object
organization            99 non-null object
organization_sector     99 non-null object
dtypes: object(8)
memory usage: 6.3+ KB


In [8]:
# Create table
c.execute('''CREATE TABLE university_professors (
firstname text,
lastname text,
university text,
university_shortname text,
university_city text,
function text,
organization text,
organization_sector text
);''')

# Save (commit) the changes
conn.commit()

In [9]:
# Insert values from uni_prof_df DataFrame into table
c.executemany('INSERT INTO university_professors VALUES (?,?,?,?,?,?,?,?)', 
              uni_prof_df.values.tolist())

# Save (commit) the changes
conn.commit()

In [10]:
c.execute('''-- Insert unique professors into the new table
INSERT INTO professors 
SELECT DISTINCT firstname, lastname, university_shortname 
FROM university_professors;''')

conn.commit()

In [11]:
for row in c.execute('SELECT * FROM professors LIMIT 5;'):
    print(row)

('Karl', 'Aberer', 'EPF')
('RezaShokrollah', 'Abhari', 'ETH')
('Georges', 'AbouJaoudé', 'EPF')
('Hugues', 'Abriel', 'UBE')
('Daniel', 'Aebersold', 'UBE')


In [12]:
# Rename a column
c.execute('''ALTER TABLE affiliations
ADD COLUMN firstname text;''')

c.execute('''ALTER TABLE affiliations
ADD COLUMN lastname text;''')

c.execute('''ALTER TABLE affiliations
ADD COLUMN function text;''')

# Save (commit) the changes
conn.commit()

In [13]:
for row in c.execute('''PRAGMA table_info('affiliations')'''):
    print(row)

(0, 'organization', 'text', 0, None, 0)
(1, 'firstname', 'text', 0, None, 0)
(2, 'lastname', 'text', 0, None, 0)
(3, 'function', 'text', 0, None, 0)


In [14]:
c.execute('''INSERT INTO affiliations 
SELECT DISTINCT firstname, lastname, function, organization 
FROM university_professors;''')

conn.commit()

In [15]:
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print(name[0])

professors
affiliations
university_professors


In [16]:
c.execute('''CREATE TABLE organizations (
organization text,
organization_sector text);''')

c.execute('''INSERT INTO organizations
SELECT DISTINCT organization, organization_sector
FROM university_professors;''')

conn.commit()

In [17]:
c.execute('''CREATE TABLE universities (
university_shortname text,
university text,
university_city text);''')

c.execute('''INSERT INTO universities
SELECT DISTINCT university_shortname, university, university_city
FROM university_professors;''')

conn.commit()

In [18]:
c.execute('DROP TABLE university_professors;')

conn.commit()

In [19]:
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print(name[0])

professors
affiliations
organizations
universities


In [20]:
for row in c.execute('''SELECT DISTINCT(university_shortname) 
FROM professors;'''):
    print(row[0])

EPF
ETH
UBE
ULA
USG
UNE
UFR


# Constraints in SQLite

- SQLite has limited ALTER commands
- Most contraints need to be specified when the column is created
- Therefore, these operations that are common in other databases like MSQL and Postgresql require the creation of a new table with the proper contstraints, copying of the data to the new table, and dropping the old table.

In [21]:
import pandas as pd
import sqlalchemy
from sqlalchemy.sql import text

In [22]:
%run keys.py

In [23]:
with engine4.connect() as con:
    a = text('''
    CREATE TABLE professors (
    firstname text,
    lastname text);
    ''')
    
    b = text('''
    ALTER TABLE professors 
    ADD COLUMN university_shortname text;
    ''')
    
    c = text('''
    CREATE TABLE affiliations (
    organisation text);
    ''')
    
    my_list = [a,b,c]
    
    for statement in my_list:
        con.execute(statement)

In [24]:
with engine4.connect() as con:
    a = text('''
    ALTER TABLE affiliations
    RENAME COLUMN organisation TO organization;
    ''')
    
    b = text('''CREATE TABLE university_professors (
    firstname text,
    lastname text,
    university text,
    university_shortname text,
    university_city text,
    function text,
    organization text,
    organization_sector text);
    ''')
    
    my_list = [a,b]
    
    for statement in my_list:
        con.execute(statement)

In [25]:
uni_prof_df.to_sql('university_professors', con=engine4, if_exists='replace')

engine4.execute("SELECT * FROM university_professors LIMIT 5").fetchall()

[(0, 'Karl', 'Aberer', 'ETHLausanne', 'EPF', 'Lausanne', 'ChairmanofL3SAdvisoryBoard', 'L3SAdvisoryBoard', 'Education&research'),
 (1, 'Karl', 'Aberer', 'ETHLausanne', 'EPF', 'Lausanne', 'MemberConseilofZeno-KarlSchindlerFoundation', 'Zeno-KarlSchindlerFoundation', 'Education&research'),
 (2, 'Karl', 'Aberer', 'ETHLausanne', 'EPF', 'Lausanne', 'MemberofConseilFondationIDIAP', 'FondationIDIAP', 'Education&research'),
 (3, 'Karl', 'Aberer', 'ETHLausanne', 'EPF', 'Lausanne', 'PanelMember', 'SNFAmbizioneProgram', 'Education&research'),
 (4, 'RezaShokrollah', 'Abhari', 'ETHZürich', 'ETH', 'Zurich', 'Aufsichtsratsmandat', 'PNEWindAG', 'Energy')]

# Useful reference for the above operation:

## [pandas.DataFrame.to_sql](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html)

In [26]:
with engine4.connect() as con:
    
    # Insert unique professors into the new table
    a = text('''
    INSERT INTO professors 
    SELECT DISTINCT firstname, lastname, university_shortname 
    FROM university_professors;
    ''')
    
    con.execute(a)
    
    re = con.execute('SELECT * FROM professors;')
    
    re_list = []
    
    for entry in re:
        re_list.append(entry)
        
    for entry in re_list[:5]:
        print(entry)

('François', 'Avellan', 'EPF')
('Marco', 'Bakker', 'EPF')
('Patrick', 'Aebischer', 'EPF')
('KayWerner', 'Axhausen', 'ETH')
('Claudio', 'Bassetti', 'UBE')


In [27]:
with engine4.connect() as con:
    
    a = text('''ALTER TABLE affiliations
    ADD COLUMN firstname text;''')
    
    b = text('''ALTER TABLE affiliations
    ADD COLUMN lastname text;''')
    
    c = text('''ALTER TABLE affiliations
    ADD COLUMN function text;''')
    
    my_list = [a,b,c]
    
    for statement in my_list:
        con.execute(statement)

In [28]:
with engine4.connect() as con:
    
    a = text('''
    INSERT INTO affiliations 
    SELECT DISTINCT organization, firstname, lastname, function 
    FROM university_professors;
    ''')
    
    b = text('''
    CREATE TABLE organizations (
    organization text,
    organization_sector text);
    ''')
    
    c = text('''
    INSERT INTO organizations
    SELECT DISTINCT organization, organization_sector
    FROM university_professors;
    ''')
    
    d = text('''
    CREATE TABLE universities (
    university_shortname text,
    university text,
    university_city text);
    ''')
    
    e = text('''
    INSERT INTO universities
    SELECT DISTINCT university_shortname, university, university_city
    FROM university_professors;
    ''')
    
    f = text('DROP TABLE university_professors;')
    
    my_list = [a,b,c,d,e,f]
    
    for statement in my_list:
        con.execute(statement)
        
    re = con.execute('SELECT DISTINCT(university_shortname) FROM professors;')
    
    print('Distinct university shortnames:')
    
    for row in re:
        print(row[0])

Distinct university shortnames:
ULA
UNE
EPF
USG
UBE
UFR
ETH


In [29]:
with engine4.connect() as con:
    
    a = text('''
    ALTER TABLE professors
    ALTER COLUMN university_shortname
    TYPE char(3);
    ''')
    
    b = text('''
    ALTER TABLE professors
    ALTER COLUMN firstname
    TYPE varchar(64);
    ''')
    
    # c = text('''
    # ALTER TABLE professors 
    # ALTER COLUMN firstname
    # TYPE varchar(16)
    # USING SUBSTRING(firstname FROM 1 FOR 16)''')
    
    c = text('''
    ALTER TABLE professors 
    ALTER COLUMN firstname SET NOT NULL;
    ''')
    
    d = text('''
    ALTER TABLE professors
    ALTER COLUMN lastname SET NOT NULL;
    ''')
    
    e = text('''
    ALTER TABLE universities
    ADD CONSTRAINT university_shortname_unq 
    UNIQUE(university_shortname);
    ''')
    
    f = text('''
    ALTER TABLE organizations
    ADD CONSTRAINT organization_unq
    UNIQUE(organization);
    ''')
    
    my_list = [a,b,c,d,e,f]
    
    for statement in my_list:
        con.execute(statement)

In [30]:
with engine4.connect() as con:
    
    re = con.execute('SELECT COUNT(*) FROM universities;')
    
    for row in re:
        print('The number of universities is:', row[0])

The number of universities is: 7


In [31]:
with engine4.connect() as con:
    
    re = con.execute('SELECT COUNT(DISTINCT(university_city)) FROM universities;')
    
    for row in re:
        print('The number of university cities is:', row[0])


The number of university cities is: 6


In [32]:
statement = '''SELECT COUNT(DISTINCT(lastname, firstname)) 
FROM professors;'''

with engine4.connect() as con:
    
    re = con.execute(statement)
    
    for row in re:
        print('The number of distinct professor names is:', row[0])

The number of distinct professor names is: 39


In [33]:
with engine4.connect() as con:
    
    a = text('''
    ALTER TABLE professors
    ADD COLUMN id serial PRIMARY KEY;

    ''')
    
    con.execute(a)

In [34]:
with engine4.connect() as con:
    
    re = con.execute('''
    -- Have a look at the first 10 rows of professors
    SELECT * FROM professors
    LIMIT 10;
    ''')
    
    for row in re:
        print(row)

('François', 'Avellan', 'EPF', 1)
('Marco', 'Bakker', 'EPF', 2)
('Patrick', 'Aebischer', 'EPF', 3)
('KayWerner', 'Axhausen', 'ETH', 4)
('Claudio', 'Bassetti', 'UBE', 5)
('Philippe', 'Bacchetta', 'ULA', 6)
('Marcelo', 'Aebi', 'ULA', 7)
('RezaShokrollah', 'Abhari', 'ETH', 8)
('Georges', 'AbouJaoudé', 'EPF', 9)
('DavidAlan', 'Basin', 'ETH', 10)


In [35]:
with engine4.connect() as con:
    
    a = text('''
    CREATE TABLE cars (
     make varchar(64) NOT NULL,
     model varchar(64) NOT NULL,
     mpg integer NOT NULL
     )
    ''')
    
    con.execute(a)

In [36]:
with engine4.connect() as con:
    
    b = text('''
    INSERT INTO cars (make, model, mpg)
    VALUES 
        ('Subaru', 'Forester', 24),
        ('Opel', 'Astra', 45),
        ('Opel', 'Vectra', 40),
        ('Ford', 'Avenger', 30),
        ('Ford', 'Galaxy', 30),
        ('Toyota', 'Prius', 50),
        ('Toyota', 'Speedster', 30),
        ('Toyota', 'Galaxy', 20),
        ('Mitsubishi', 'Forester', 10),
        ('Mitsubishi', 'Galaxy', 30);
    ''')
    
    con.execute(b)

In [37]:
with engine4.connect() as con:
    
    re = con.execute('SELECT * FROM cars')
    
    for row in re:
        print(row)

('Subaru', 'Forester', 24)
('Opel', 'Astra', 45)
('Opel', 'Vectra', 40)
('Ford', 'Avenger', 30)
('Ford', 'Galaxy', 30)
('Toyota', 'Prius', 50)
('Toyota', 'Speedster', 30)
('Toyota', 'Galaxy', 20)
('Mitsubishi', 'Forester', 10)
('Mitsubishi', 'Galaxy', 30)


In [38]:
def run_query(statement):
    with engine4.connect() as con:
        a = text(statement)
        
        con.execute(a)

In [39]:
def run_query_print_rows(statement):
    with engine4.connect() as con:
        
        re = con.execute(statement)
        
        for row in re:
            print(row)

In [40]:
statement = '''
-- Count the number of distinct rows with columns make, model
SELECT COUNT(DISTINCT(make, model)) 
FROM cars;

-- Add the id column
ALTER TABLE cars
ADD COLUMN id varchar(128);

-- Update id with make + model
UPDATE cars
SET id = CONCAT(make, model);

-- Make id a primary key
ALTER TABLE cars
ADD CONSTRAINT id_pk PRIMARY KEY(id);
'''

run_query(statement)

In [41]:
statement = '''
-- Have a look at the table
SELECT * FROM cars;
'''

run_query_print_rows(statement)

('Subaru', 'Forester', 24)
('Opel', 'Astra', 45)
('Opel', 'Vectra', 40)
('Ford', 'Avenger', 30)
('Ford', 'Galaxy', 30)
('Toyota', 'Prius', 50)
('Toyota', 'Speedster', 30)
('Toyota', 'Galaxy', 20)
('Mitsubishi', 'Forester', 10)
('Mitsubishi', 'Galaxy', 30)


In [42]:
s = '''
-- Create the table
CREATE TABLE students (
  last_name varchar(128) NOT NULL,
  ssn integer PRIMARY KEY,
  phone_no char(12)
);
'''

run_query(s)

In [43]:
s = '''
ALTER TABLE universities
RENAME COLUMN university_shortname TO id;

ALTER TABLE universities
ADD CONSTRAINT id_pkey PRIMARY KEY(id);
'''

run_query(s)

In [44]:
s = '''
ALTER TABLE professors
RENAME COLUMN university_shortname TO university_id;

ALTER TABLE professors 
ADD CONSTRAINT professors_fkey FOREIGN KEY (university_id) REFERENCES universities (id);
'''

run_query(s)

In [45]:
s = '''
-- Try to insert a new professor
INSERT INTO professors (firstname, lastname, university_id)
VALUES ('Albert', 'Einstein', 'UBE');
'''

run_query(s)

In [46]:
s = '''
-- Select all professors working for universities in the city of Zurich
SELECT professors.lastname, universities.id, universities.university_city
FROM professors
JOIN universities
ON professors.university_id = universities.id
WHERE universities.university_city = 'Zurich';
'''

run_query_print_rows(s)

('Axhausen', 'ETH', 'Zurich')
('Abhari', 'ETH', 'Zurich')
('Basin', 'ETH', 'Zurich')
('Baschera', 'ETH', 'Zurich')
('Bechtold', 'ETH', 'Zurich')


In [47]:
s = '''
ALTER TABLE organizations
RENAME organization to id;
'''

run_query(s)

In [54]:
s = '''
ALTER TABLE affiliations
ADD COLUMN professor_id integer REFERENCES professors (id);

ALTER TABLE affiliations
RENAME organization TO organization_id;

ALTER TABLE affiliations
ADD CONSTRAINT affiliations_organization_fkey FOREIGN KEY (organization_id) REFERENCES organizations (id);
'''

run_query(s)

In [55]:
s = '''
SELECT * FROM affiliations LIMIT 10
'''

run_query_print_rows(s)

('St.GallerKantonalbankAG', 'Manuel', 'Ammann', 'Verwaltungsrat', None)
('cleanenergyresearchprogram', 'Christophe', 'Ballif', 'Panel', None)
('NestecSA', 'David', 'AtienzaAlonso', 'Consultancyonembeddedsystemsusesforswallowingandchewingmonitoring', None)
('Nestlé', 'Patrick', 'Aebischer', 'Verwaltungsrat', None)
('SWANIsotopenAG', 'Daniel', 'Aebersold', None, None)
('SchweizerischeGesellschaftfürallgemeineinnereMedizin', 'Drahomir', 'Aujesky', None, None)
('AlgofinAG', 'Manuel', 'Ammann', 'Verwaltungsratspräsident', None)
('FondazioneEccles', 'Claudio', 'Bassetti', None, None)
('InternationalSchoolofBern', 'Artur', 'Baldauf', None, None)
('BernischeStiftungfürkrebskrankeKinderundJugendliche', 'Christoph', 'Aebi', None, None)


In [60]:
s = '''
UPDATE affiliations
SET professor_id = professors.id
FROM professors
WHERE affiliations.firstname = professors.firstname AND affiliations.lastname = professors.lastname;
'''

run_query(s)

In [61]:
s = '''
SELECT * FROM affiliations LIMIT 10
'''

run_query_print_rows(s)

("NationalerGeistigerRatderBahá'íderSchweiz", 'Omid', 'Aschari', 'Mitglied', 15)
('SingaporeInternationalAdvisoryCommitteeforBiomedicalSciences', 'Patrick', 'Aebischer', 'Beirat', 3)
('SpitalThunAG', 'Iris', 'Baumgartner', None, 11)
('InternationalJournalofNumberTheory', 'Eva', 'BayerFluckiger', 'Membreducomitééditorial', 16)
('HiltiAG', 'PiusEliseo', 'Baschera', 'VR-Präsidium', 28)
('NeueBankAG', 'Manuel', 'Ammann', 'Verwaltungsrat', 37)
('gyMetricsSA', 'Yann', 'Barrandon', "Conseild'administration", 18)
('L3SAdvisoryBoard', 'Karl', 'Aberer', 'ChairmanofL3SAdvisoryBoard', 23)
('St.GallerKantonalbankAG', 'Manuel', 'Ammann', 'Verwaltungsrat', 37)
('cleanenergyresearchprogram', 'Christophe', 'Ballif', 'Panel', 33)


In [63]:
s = '''
ALTER TABLE affiliations
DROP COLUMN firstname;

ALTER TABLE affiliations
DROP COLUMN lastname;
'''

run_query(s)

# Referential Integrity

## A record referencing another table must refer to an existing record in that table

- Specified between two tables
- Enforced through foreign keys

## Referential integrity violations
#### Referential integrity from table A to table B is violated...

- ...if a record in table B that is referenced from a record in table A is deleted.
- ...if a record in table A referencing a non-existing record from table B is inserted.
#### Foreign keys prevent violations!

## Dealing with violations

```sql
CREATE TABLE a (
 id integer PRIMARY KEY,
 column_a varchar(64), 
 ...,
 b_id integer REFERENCES b (id) ON DELETE NO ACTION
);
```

```sql
CREATE TABLE a (
 id integer PRIMARY KEY,
 column_a varchar(64), 
 ...,
 b_id integer REFERENCES b (id) ON DELETE CASCADE
);
```

## Dealing with violations, contd.
#### ON DELETE...

- ...NO ACTION: Throw an error
- ...CASCADE: Delete all referencing records
- ...RESTRICT: Throw an error
- ...SET NULL: Set the referencing column to NULL
- ...SET DEFAULT: Set the referencing column to its default value

In [65]:
s = '''
SELECT constraint_name, table_name, constraint_type
FROM information_schema.table_constraints
WHERE constraint_type = 'FOREIGN KEY';
'''

run_query_print_rows(s)

('professors_fkey', 'professors', 'FOREIGN KEY')
('affiliations_professor_id_fkey', 'affiliations', 'FOREIGN KEY')
('affiliations_organization_fkey', 'affiliations', 'FOREIGN KEY')


In [67]:
s = '''
ALTER TABLE affiliations
DROP CONSTRAINT affiliations_organization_fkey;

ALTER TABLE affiliations
ADD CONSTRAINT affiliations_organization_id_fkey FOREIGN KEY (organization_id) REFERENCES organizations (id) ON DELETE CASCADE;

DELETE FROM organizations 
WHERE id = 'CUREM';

SELECT * FROM organizations
WHERE id = 'CUREM';
'''

run_query(s)

In [68]:
s = '''
SELECT COUNT(*), professors.university_id 
FROM affiliations
JOIN professors
ON affiliations.professor_id = professors.id
GROUP BY professors.university_id 
ORDER BY count DESC;
'''

run_query_print_rows(s)

(51, 'EPF')
(15, 'UBE')
(14, 'USG')
(10, 'ETH')
(5, 'ULA')
(3, 'UNE')
(1, 'UFR')


In [70]:
s = '''
SELECT COUNT(*), organizations.organization_sector, 
professors.id, universities.university_city
FROM affiliations
JOIN professors
ON affiliations.professor_id = professors.id
JOIN organizations
ON affiliations.organization_id = organizations.id
JOIN universities
ON professors.university_id = universities.id
WHERE organizations.organization_sector = 'Media&communication'
GROUP BY organizations.organization_sector, 
professors.id, universities.university_city
ORDER BY count DESC;
'''

run_query_print_rows(s)

(3, 'Media&communication', 16, 'Lausanne')
(1, 'Media&communication', 32, 'Lausanne')
